# Data Collection and Cleaning: BABIP, Hit Distribution, and Statcast Relationships

Group 15, ID: f7469

- John Dunlevy (dunlevy.j@northeastern.edu)
- Justin Chen (chen.justi@northeastern.edu)

## Data

### Overview

We will obtain 2 sets of data: player's basic statistics as well as their statcast game logs. We chose to use a list of qualified hitters in 2019 (meaning they reached 3.1 at bats per team game played). That list of players for the 2019 season can be found [here](https://baseballsavant.mlb.com/leaderboard/custom?year=2019).

Batting average on balls in play, or BABIP, is equal to: (hits - homeruns) / (at-bats - homeruns - srikeouts + sacrifice-flies). Essentially, it is the percentage of balls that a batter hits in play that result in a base hit (in this case a single, double, or triple). In order to be considered "in play", the ball must be hit by the batter and not leave the playing field. Thus, homeruns and foul balls that enter the stands are not included. 


Statcast uses doppler radar and HD video to measure statistics such as pitch speed and spin rate, batted ball speed, launch angle, sprint speed, and more, and its data is available from every MLB game since the start of the 2015 season. Baseball Savant is used to get MLB player game logs which includes Statcast (exit velocity, launch angle), and hit result data, as well as basic statistics like batting average, base hits, and on-base percentage. Individual game logs are available for nearly all games during the 2019 season. In particular, for each player, information about each plate appearance from each game in the 2019 season are available on Baseball Savant. 

#### Basic Statistics
For the player's basic statistics, we were able to export data directly from Savant to a csv file:
<img src="https://snipboard.io/vzk9PH.jpg" width=800 px>

We downloaded the default statistics along with:
- different types of base hits (singles, doubles, triples, homeruns)
- On-base percentage
- Sac Flies
- walk rate (percentage of plate appearances resulting in a walk)
- strikeout rate (percentage of plate appearances resulting in a strikeout)
- Slugging percentage

... and more 

to have a complete dataset at our disposal as well as the statistics needed to calculate BABIP.


##### Pipeline Overview
We will accomplish this task with these functions:
- `get_name()` 
    - returns a player's full name
- `get_player_babip()`
    - calculates the BABIP based on a player's statistical data
    
As well as these scripts:
- **Building a player dict**: using `get_name()` to find a full list of players with qualified at-bats
- **Add BABIP**: uses `get_player_babip()` to calculate BABIP for each qualified hitter and add to the dataframe of basic stats

#### Game logs
Next, we seek to obtain each player's game logs by scraping Baseball Savant's gamelog datasets since they couldn't be directly downloaded. An example game log can be found [here](https://baseballsavant.mlb.com/savant-player/juan-soto-665742?stats=gamelogs-r-hitting-statcast&season=2020) <img src = "https://snipboard.io/NcOtge.jpg" width = 800 px>

From the game logs, we scraped:
- exit velocity
- launch angle
- result

to complete a dataset of each hit ball (as well as it's Statcast data) of each qualified hitter

##### Pipeline Overview
We will accomplish this task with these functions:
- `get_player_url()` 
    - returns a url formatted for given player
- `get_player_url_dict()`
    - gets a dict containing all urls of qualified hitters in 2019
- `get_player_gamelogs()`
    - finds the game logs for each individual player
- `is_ball_in_play()`
    - checks to see if a ball is in play
- `get_exit_velo_idx()`
    - gets the exit velocity from the game log
- `clean_player_gamelogs()`
    - cleans up and formats the player game logs and extracts the desired data using `is_ball_in_play()` and `get_exit_velo_idx()`

As well as these scripts:
- **Scraping and formatting Savant**: using the functions listed above to scrape and clean to obtain a dataset of each player's gamelog 
- **Add missing players**: because some player's have unique or repeated names, 3 of them (J.D. Martinez, J.T. Realmuto, Josh Bell) had to be manually written in

### MLB IDs
In order to construct URLs to scrape game log data from Baseball Savant, we need to match players to their `mlb_id`, a unique 6-digit number used to identify players in mlb statistical data bases. We obtain the following data set from [Crunch Time Baseball](http://crunchtimebaseball.com/baseball_map.html), which contains `mlb_id`s and ids for other statistical websites for current and historical players. 

In [1]:
import pandas as pd 

# Read 'mlb_ids.csv' which contains ids for mlb players on baseball savant
df_mlb_ids = pd.read_csv('mlb_ids.csv', encoding='latin-1')

In [2]:
df_mlb_ids.head()

,mlb_id,mlb_name,mlb_pos,mlb_team,mlb_team_long,bats,throws,birth_date,bp_id,bref_id,...,retro_name,debut,yahoo_id,yahoo_name,ottoneu_id,ottoneu_name,ottoneu_pos,rotowire_id,rotowire_name,rotowire_pos
0,592091,A.J. Achter,P,LAA,Los Angeles Angels,R,R,8/27/1988,66919.0,achteaj01,...,A.J. Achter,9/3/2014,9824.0,A.J. Achter,18093.0,A.J. Achter,RP,NaN,NaN,NaN
1,595918,A.J. Cole,P,TOR,Toronto Blue Jays,R,R,1/5/1992,68086.0,coleaj01,...,A.J. Cole,4/28/2015,9638.0,A.J. Cole,14940.0,A.J. Cole,RP,11446.0,A.J. Cole,P
2,454560,A.J. Ellis,C,SD,San Diego Padres,R,R,4/9/1981,47564.0,ellisaj01,...,A.J. Ellis,9/15/2008,8373.0,A.J. Ellis,1543.0,A.J. Ellis,Util,9242.0,A.J. Ellis,C
3,456167,A.J. Griffin,P,TEX,Texas Rangers,R,R,1/28/1988,65757.0,griffaj01,...,A.J. Griffin,6/24/2012,9220.0,A.J. Griffin,13764.0,A.J. Griffin,RP,12410.0,A.J. Griffin,P
4,543362,A.J. Jimenez,C,TEX,Texas Rangers,R,R,5/1/1990,58320.0,jimenaj01,...,A.J. Jimenez,9/6/2017,NaN,NaN,17777.0,A.J. Jimenez,Util,12071.0,A.J. Jimenez,C


### Building player dict

Below is the initial basic stats DataFrame (`df_basic_stats`) downloaded from [Baseball Savant](https://baseballsavant.mlb.com/leaderboard/custom?year=2019&type=batter&filter=&sort=4&sortDir=desc&min=q&selections=player_age,b_ab,b_total_pa,b_total_hits,b_single,b_double,b_triple,b_home_run,b_strikeout,b_walk,b_k_percent,b_bb_percent,batting_avg,slg_percent,on_base_percent,on_base_plus_slg,b_sac_fly,xba,xslg,woba,xwoba,xobp,xiso,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,&chart=false&x=xba&y=xba&r=no&chartType=beeswarm) and includes the "Basic Stats" filter as well as Sacrifice Flies for qualified hitters in the 2019 season. 

In [3]:
# Read 'stats2019.csv' as df_basic_stats, containing basic statistics for qualified hitters in 2019
df_basic_stats_2019 = pd.read_csv('stats2019.csv')
del df_basic_stats_2019['Unnamed: 29']
df_basic_stats_2019

,last_name,first_name,year,player_age,b_ab,b_total_pa,b_total_hits,b_single,b_double,b_triple,...,b_sac_fly,xba,xslg,woba,xwoba,xobp,xiso,exit_velocity_avg,launch_angle_avg,barrel_batted_rate
0,Pujols,Albert,2019,39,491,545,120,75,22,0,...,8,0.247,0.419,0.308,0.314,0.310,0.172,88.3,12.5,5.8
1,Cabrera,Miguel,2019,36,493,549,139,106,21,0,...,5,0.266,0.433,0.318,0.328,0.334,0.167,90.4,12.3,6.7
2,Choo,Shin-Soo,2019,37,563,660,149,92,31,2,...,1,0.262,0.473,0.353,0.361,0.369,0.212,91.9,9.1,9.0
3,Jones,Adam,2019,34,485,528,126,84,25,1,...,3,0.256,0.427,0.309,0.317,0.311,0.171,86.1,13.4,6.7
4,Cruz,Nelson,2019,39,454,521,141,74,26,0,...,3,0.290,0.636,0.420,0.416,0.378,0.346,93.7,13.1,19.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Acuna Jr.,Ronald,2019,22,626,715,175,110,22,2,...,1,0.283,0.577,0.370,0.393,0.368,0.294,90.6,14.2,15.0
131,Fletcher,David,2019,25,596,653,173,133,30,4,...,1,0.288,0.366,0.319,0.316,0.348,0.078,83.8,9.9,0.2
132,Guerrero Jr.,Vladimir,2019,20,464,514,126,83,26,2,...,2,0.256,0.429,0.329,0.327,0.326,0.173,89.4,6.7,7.7
133,Soto,Juan,2019,21,542,659,153,82,32,5,...,6,0.285,0.574,0.394,0.408,0.404,0.290,92.0,12.5,12.3


Repeat for 2020: 

In [4]:
df_basic_stats_2020 = pd.read_csv('stats2020.csv')
del df_basic_stats_2020['Unnamed: 30']
df_basic_stats_2020

,last_name,first_name,player_id,year,player_age,b_ab,b_total_pa,b_total_hits,b_single,b_double,...,b_sac_fly,xba,xslg,woba,xwoba,xobp,xiso,exit_velocity_avg,launch_angle_avg,barrel_batted_rate
0,Peralta,David,444482,2020,33,203,218,61,45,10,...,2,0.257,0.367,0.333,0.295,0.301,0.110,89.2,6.4,5.0
1,Santana,Carlos,467793,2020,34,206,255,41,26,7,...,1,0.250,0.444,0.316,0.368,0.390,0.194,88.0,12.2,6.7
2,Brantley,Michael,488726,2020,33,170,187,51,31,15,...,0,0.260,0.413,0.362,0.325,0.327,0.153,88.7,10.2,4.9
3,Escobar,Eduardo,500871,2020,31,203,222,43,29,7,...,2,0.261,0.394,0.257,0.305,0.316,0.132,88.6,18.1,5.5
4,Gonzalez,Marwin,503556,2020,31,175,199,37,28,4,...,4,0.240,0.401,0.269,0.314,0.315,0.162,89.2,12.6,5.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Bellinger,Cody,641355,2020,25,213,243,51,29,10,...,0,0.280,0.487,0.340,0.372,0.369,0.207,89.3,16.6,9.4
138,Tucker,Kyle,663656,2020,23,209,228,56,29,12,...,1,0.265,0.453,0.349,0.331,0.322,0.188,91.1,14.9,9.1
139,Happ,Ian,664023,2020,26,198,231,51,27,11,...,0,0.258,0.462,0.369,0.358,0.361,0.204,91.1,9.0,10.3
140,Tatis Jr.,Fernando,665487,2020,21,224,257,62,32,11,...,1,0.296,0.608,0.392,0.419,0.385,0.312,95.9,8.7,19.5


#### Updating Players' Names
The `get_name(player)` function takes a row from `df_basic_stats` and returns a single string for the players name. This is used to make the player's names uniform across multiple datasets. 

In [5]:
def get_name(player):
    ''' Gets a player's name from a row in df_basic_stats
    
    Args:
        player (pd.Series): row representing a player's info/stats in df_basic_stats
        
    Returns:
        name (str): player's full name in a single string
    
    '''
    # gets the first and last name from the columns, 
    # combines first and last name and remove leading space 
    name = player[' first_name'] + ' ' + player['last_name']
    name = name[1:]
    
    return name
    

#### Obtaining Baseball Savant Player Codes
The following script creates a dictionary containing `mlb_id` and `name` pairs that pair players with their id on Baseball Savant. The id's are used to generate each player's link containing Statcast gamelog data. 

In [6]:
# Add mlb_id to df_basic_stats 
def get_player_ids(df_stats):
    ''' Adds column containing mlb_id of each player in the given DataFrame 
    
    Args:
        df_stats (pd.DataFrame): season statistics for qualified players 
    
    Returns:
        player_id_dict (dict): contains player name, mlb_id pairs
    '''
    # Empty dict to contain id, player name pairs 
    player_id_dict = {}

    # For each qualified hitter, map name to id 
    for idx, player in df_stats.iterrows():
        # Combine player first and last name, remove leading space
        name = get_name(player)
        if name == 'Michael Brantley Jr.':
            name = 'Michael Brantley'
            
        # Boolean mapping to find row in ids df corresponding to the player
        id_rows = df_mlb_ids['mlb_name'] == name
        df_id_name = df_mlb_ids.loc[id_rows,:]
        player_id_dict[int(df_id_name.iloc[0,:]['mlb_id'])] = name

    return player_id_dict

In [7]:
# Get player_id_dict for 2019, 2020 seasons
player_id_dict_2019 = get_player_ids(df_basic_stats_2019)
player_id_dict_2020 = get_player_ids(df_basic_stats_2020)

# Add player's full name as a column in df_basic_stats, remove first and last name columns 
df_basic_stats_2019['name'] = player_id_dict_2019.values()
df_basic_stats_2020['name'] = player_id_dict_2020.values()


del df_basic_stats_2019['last_name']
del df_basic_stats_2019[' first_name']
del df_basic_stats_2020['last_name']
del df_basic_stats_2020[' first_name']

# Add mlb_id as column in df_basic_stats 
df_basic_stats_2019['mlb_id'] = player_id_dict_2019
df_basic_stats_2020['mlb_id'] = player_id_dict_2020
df_basic_stats_2019.head()

,year,player_age,b_ab,b_total_pa,b_total_hits,b_single,b_double,b_triple,b_home_run,b_strikeout,...,xslg,woba,xwoba,xobp,xiso,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,name,mlb_id
0,2019,39,491,545,120,75,22,0,23,68,...,0.419,0.308,0.314,0.310,0.172,88.3,12.5,5.8,Albert Pujols,405395
1,2019,36,493,549,139,106,21,0,12,108,...,0.433,0.318,0.328,0.334,0.167,90.4,12.3,6.7,Miguel Cabrera,408234
2,2019,37,563,660,149,92,31,2,24,165,...,0.473,0.353,0.361,0.369,0.212,91.9,9.1,9.0,Shin-Soo Choo,425783
3,2019,34,485,528,126,84,25,1,16,101,...,0.427,0.309,0.317,0.311,0.171,86.1,13.4,6.7,Adam Jones,430945
4,2019,39,454,521,141,74,26,0,41,131,...,0.636,0.420,0.416,0.378,0.346,93.7,13.1,19.9,Nelson Cruz,443558


In [8]:
df_basic_stats_2020.head()

,player_id,year,player_age,b_ab,b_total_pa,b_total_hits,b_single,b_double,b_triple,b_home_run,...,xslg,woba,xwoba,xobp,xiso,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,name,mlb_id
0,444482,2020,33,203,218,61,45,10,1,5,...,0.367,0.333,0.295,0.301,0.110,89.2,6.4,5.0,David Peralta,444482
1,467793,2020,34,206,255,41,26,7,0,8,...,0.444,0.316,0.368,0.390,0.194,88.0,12.2,6.7,Carlos Santana,467793
2,488726,2020,33,170,187,51,31,15,0,5,...,0.413,0.362,0.325,0.327,0.153,88.7,10.2,4.9,Michael Brantley,488726
3,500871,2020,31,203,222,43,29,7,3,4,...,0.394,0.257,0.305,0.316,0.132,88.6,18.1,5.5,Eduardo Escobar,500871
4,503556,2020,31,175,199,37,28,4,0,5,...,0.401,0.269,0.314,0.315,0.162,89.2,12.6,5.1,Marwin Gonzalez,503556


#### Calculating a Player's BABIP

In [9]:
def get_player_babip(player, df_basic_stats):
    ''' Calculate's a player's batting average on balls in play (BABIP)
    babip = (hits - homeruns) / (ab - strikeouts - homeruns + sac_fly)
    
    Args:
        player (str): Player's name
        df_basic_stats (pd.DataFrame): stats for qualified hitters in a season
        
    Returns:
        babip (float): player's BABIP
    
    '''
    
    bool_player = df_basic_stats['name'] == player
    df_player = df_basic_stats.loc[bool_player]
    
    hits = df_player['b_total_hits']
    home_runs = df_player['b_home_run']
    at_bats = df_player['b_ab']
    strikeouts = df_player['b_strikeout']
    sac_flies = df_player['b_sac_fly']

    #calculates babip for each player with it's formula
    babip = float((hits - home_runs) / (at_bats - strikeouts - home_runs + sac_flies))
    
    return babip

In [10]:
def get_all_players_babip(player_id_dict, df_stats):
    ''' Returns a dictionary with each player's babip based on the given stats
    
    Args:
        player_id_dict (dict): mlb_id, player name pairs
        df_stats (pd.DataFrame): batting statistics for all qualified hitters in a season
        
    Returns:
        dict_babip (dict): player, babip pairs
    
    '''

    dict_babip = {}

    for player in player_id_dict.values():
        #calculates all player's babip
        dict_babip[player] = get_player_babip(player, df_stats)
    
    return dict_babip

#### Add BABIP to df_basic_stats

In [11]:
babip_dict_2019 = get_all_players_babip(player_id_dict_2019, df_basic_stats_2019)
df_basic_stats_2019['babip'] = babip_dict_2019.values()

babip_dict_2020 = get_all_players_babip(player_id_dict_2020, df_basic_stats_2020)
df_basic_stats_2020['babip'] = babip_dict_2020.values()

In [12]:
df_basic_stats_2019.head()

,year,player_age,b_ab,b_total_pa,b_total_hits,b_single,b_double,b_triple,b_home_run,b_strikeout,...,woba,xwoba,xobp,xiso,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,name,mlb_id,babip
0,2019,39,491,545,120,75,22,0,23,68,...,0.308,0.314,0.310,0.172,88.3,12.5,5.8,Albert Pujols,405395,0.237745
1,2019,36,493,549,139,106,21,0,12,108,...,0.318,0.328,0.334,0.167,90.4,12.3,6.7,Miguel Cabrera,408234,0.335979
2,2019,37,563,660,149,92,31,2,24,165,...,0.353,0.361,0.369,0.212,91.9,9.1,9.0,Shin-Soo Choo,425783,0.333333
3,2019,34,485,528,126,84,25,1,16,101,...,0.309,0.317,0.311,0.171,86.1,13.4,6.7,Adam Jones,430945,0.296496
4,2019,39,454,521,141,74,26,0,41,131,...,0.420,0.416,0.378,0.346,93.7,13.1,19.9,Nelson Cruz,443558,0.350877


In [13]:
df_basic_stats_2020.to_csv('basic_2020_babip.csv')
df_basic_stats_2020.head()

,player_id,year,player_age,b_ab,b_total_pa,b_total_hits,b_single,b_double,b_triple,b_home_run,...,woba,xwoba,xobp,xiso,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,name,mlb_id,babip
0,444482,2020,33,203,218,61,45,10,1,5,...,0.333,0.295,0.301,0.110,89.2,6.4,5.0,David Peralta,444482,0.361290
1,467793,2020,34,206,255,41,26,7,0,8,...,0.316,0.368,0.390,0.194,88.0,12.2,6.7,Carlos Santana,467793,0.211538
2,488726,2020,33,170,187,51,31,15,0,5,...,0.362,0.325,0.327,0.153,88.7,10.2,4.9,Michael Brantley,488726,0.335766
3,500871,2020,31,203,222,43,29,7,3,4,...,0.257,0.305,0.316,0.132,88.6,18.1,5.5,Eduardo Escobar,500871,0.243750
4,503556,2020,31,175,199,37,28,4,0,5,...,0.269,0.314,0.315,0.162,89.2,12.6,5.1,Marwin Gonzalez,503556,0.240602


## Web-Scraping Baseball Savant Statcast game logs

#### Generating Player URLs 
Rather than creating a function to generate the entire URL for a player, we decided to only generate the name portion of the URL to make testing name edge cases easier to read. 

In [14]:
def get_player_url_name(player_name):
    ''' Converts player's name into url name format. See test cases for examples
    
    Args:
        player_name (str): player's name (First_Name Last_Name Suffix (if applicable))
    
    Returns:
        url_name (str): player's name in url format (first_name-last_name-suffix), all lowercase
    '''
    #find player names and formats to the url
    name_list = player_name.split()
    url_name = '-'.join(name_list)
    
    #replace any periods in names (ex J.D.) with empty strings
    url_name = url_name.replace('.', '')
    url_name = url_name.lower()
    
    return url_name

In [15]:
assert get_player_url_name('Albert Pujols') == 'albert-pujols'

# Note "Jr./Sr." treated just like first name and last name 
assert get_player_url_name('Jackie Bradley Jr.') == 'jackie-bradley-jr'

In [16]:
def get_player_url_dict(df_players, year):
    ''' Creates a dictionary containing urls to Baseball Savant statcast game logs for
    each player in the given DataFrame for the given year
    
    Args:
        df_players (pd.DataFrame): contains stats and baseball savant id for each player 
        year (int): year 
        
    Returns:
        player_url_dict (dict): dict containing player name and 2019 Baseball Savant statcast
            gamelogs for each player 
    
    '''
    # Start and end url which is the same for each player
    start_url = 'https://baseballsavant.mlb.com/savant-player/'
    end_url = f'?stats=gamelogs-r-hitting-statcast&season={year}&gamelogs_event=&gamelogs_direction=&gamelogs_pitchtypes=&gamelogs_view=statcastGameLogs'

    player_url_dict = {}
    
    # For each player, add player name and url pair to dict
    for idx, player in df_players.iterrows():
        # Combine player first/last name, remove leading space
        name = player['name']
        
        # Convert player name to url format, add '-{mlb_id}', combine parts of the url 
        player_name_url = get_player_url_name(name) + '-' + str(player['mlb_id'])
        url = start_url + player_name_url + end_url
        
        # Add name, url pair to dict
        player_url_dict[name] = url
        
    return player_url_dict

#### Scraping Data with Selenium

In [17]:
#scrapes the Baseball Savant Database for individual game logs for each player
#dataframe is later saved to a csv


from selenium import webdriver
def get_player_gamelogs(player_url):
    ''' Scrapes the given player's url to obtain the rows each representing a plate apperance in Statcast game logs. 
    
    Args:
        player_url (str): url for webpage containing the player's Statcast gamelogs 
        
    Returns:
        player_game_logs (WebElement): contains list of Statcast gamelog entries 
    '''
   
    driver = webdriver.Chrome()
    driver.get(player_url)
    
    gamelog_table = driver.find_element_by_id('gamelogs_statcast')
    
    body = gamelog_table.find_element_by_xpath('table/tbody')
    
    player_game_logs = gamelog_table.find_elements_by_class_name('default-table-row')
    
    
    return player_game_logs
    


The function `is_ball_in_play(result)` detrmines if a plate appearance resulted in a batted ball in play. If not, the plate appearance is excluded from the game logs DataFrame that we compile. 

In [18]:
def is_ball_in_play(result):
    ''' Determines if the given plate appearance result is a corresponds to
    a batted ball in play
    
    Args:
        result (str): result of the plate appearance
        
    Returns:
        is_ball_in_play (bool): whehter or not the result was a ball in play 
    '''
    
    #ignore cases that do not contribute to babip
    not_ball_in_play = ['strikeout', 'walk', 'home_run', 'hit_by_pitch', 'catcher_interf']
    
    is_ball_in_play = result not in not_ball_in_play
    
    return is_ball_in_play

##### Finding exit velocity index
The function `get_exit_velo_idx` returns the index of the exit velocity in the given gamelog string. For every batted ball event considered, the exit velocity is the first numeric element in the game log. Because the `game_log` string is split by spaces and some pitchers have 2 spaces within their name (e.g. Hyun Jin Ryu), the index of the exit velocity in the split list is not always the same. From the `exit_velo_idx`, the indices of all other relevant statcast values can be determined.  

In [19]:
def get_exit_velo_idx(game_log):
    ''' Returns the index containing exit velocity from the given game log entry
    
    Args:
        game_log (str): game log containing statistics about a batted ball event
        
    Returns:
        exit_velo_idx (int): index containing exit velocity of the batted ball in the game log 
    '''
    
    exit_velo_idx = -1
    try_idx = 5  # 5 is the lowest possible index 
    
    # Until the first float is found, increase idx 
    while (exit_velo_idx == -1):
        try:
            float(game_log[try_idx])
            exit_velo_idx = try_idx
        except:
            try_idx += 1
        
    return exit_velo_idx

In [20]:
def clean_player_gamelogs(gamelogs, player):
    '''clean up the game logs and extract name, result, and hit data for each plate appearance in a single player's
    season
    
    Args:
        gamelogs(df): dataframe of all player's game logs
        player(str): player name
    
    Returns:
        df_player(df): dataframe containing data (mentioned above) for desired player
    
    
    '''

    df_player = pd.DataFrame()
    
    for pa in gamelogs:
        # Split the gamelog string by spaces to separate entires
        pa_list = pa.text.split()
        
        # If the length of the list is not greater than 12, a batted ball did not occur or 
        # there is no statcast data available
        if len(pa_list) >= 12:
            # Get the index of the exit velocity
            ev_idx = get_exit_velo_idx(pa_list)
            
            # Get the result of the gamelog 
            result = pa_list[ev_idx - 1]
            
            if is_ball_in_play(result):
                dict_battedball = {}
                #extracts relevant data
                dict_battedball['player'] = player
                dict_battedball['result'] = result
                dict_battedball['exit_velo'] = float(pa_list[ev_idx])
                dict_battedball['launch_angle'] = int(pa_list[ev_idx + 1])
                
                # Append the row to the cleaned DataFrame 
                df_player = df_player.append(dict_battedball, ignore_index=True)
            
    return df_player

### Scraping Script:
Because the Baseball Savant webpage we are using is dynamic, BeautifulSoup cannot be used to scrape the data. Thus, selenium is used to open the webpage and extract the relevant information. Because each webpage must open and fully load before the data can be collected, the script takes a significant amount of time (> 30 minutes). 

Thus, the script was run once and the resulting DataFrame was saved to a csv which is then loaded into the notebook on subsequent runs to reduce the amount of times the script must be run. 

In [21]:
#scrapes the Baseball Savant Database for individual game logs for each player
#dataframe is later saved to a csv
#this chunk is commented out to shorten runtime (reads the saved csv instead)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

'''
# Get dictionary of player urls 
player_urls_dict = get_player_url_dict(df_basic_stats_2020, 2020)


df_players_gamelogs = pd.DataFrame()
for player in player_urls_dict.keys():
    driver = webdriver.Chrome()
    # Wait 15 seconds to allow page to load
    driver.implicitly_wait(15)
    
    driver.get(player_urls_dict[player])
    
    # Find gamelogs_statcast table, and get each row in the table 
    gamelog_table = driver.find_element_by_id('gamelogs_statcast')
    player_game_logs = gamelog_table.find_elements_by_class_name('default-table-row')
    
    # Clean the gamelogs 
    df_player_gamelogs_clean = clean_player_gamelogs(player_game_logs, player)
    
    # Append the cleaned dataframe of the player's gamelogs to the end of the cumulative dataframe 
    df_players_gamelogs = df_players_gamelogs.append(df_player_gamelogs_clean, ignore_index=True)
    
'''

#df_gamelogs_from_csv = df_players_gamelogs

#df_players_gamelogs.to_csv('gamelogs2020.csv')


"\n# Get dictionary of player urls \nplayer_urls_dict = get_player_url_dict(df_basic_stats_2020, 2020)\n\n\ndf_players_gamelogs = pd.DataFrame()\nfor player in player_urls_dict.keys():\n    driver = webdriver.Chrome()\n    # Wait 15 seconds to allow page to load\n    driver.implicitly_wait(15)\n    \n    driver.get(player_urls_dict[player])\n    \n    # Find gamelogs_statcast table, and get each row in the table \n    gamelog_table = driver.find_element_by_id('gamelogs_statcast')\n    player_game_logs = gamelog_table.find_elements_by_class_name('default-table-row')\n    \n    # Clean the gamelogs \n    df_player_gamelogs_clean = clean_player_gamelogs(player_game_logs, player)\n    \n    # Append the cleaned dataframe of the player's gamelogs to the end of the cumulative dataframe \n    df_players_gamelogs = df_players_gamelogs.append(df_player_gamelogs_clean, ignore_index=True)\n    \n"

#### Add missing players

In [22]:
def find_missing_players(df_stats, df_gamelogs):
    '''Finds players in df_stats that are not in df_gamelogs
    
    Args:
        df_stats (pd.DataFrame): basic stats for qualified players
        df_gamelogs (pd.DataFrame): statcast gamelogs for players in the same season
        
    Returns:
        missing_players (list): list of players in df_stats that do not appear in df_gamelogs
    '''
    # Initial empty list of missing players 
    missing_players = []
    
    # Get list of players that appear in gamelogs 
    gamelogs_unique = df_gamelogs['player'].unique()
    
    # For each player in df_stats, if not in gamelogs_unique, add to missing_players
    for idx, player in df_stats.iterrows():
        if player['name'] not in gamelogs_unique:
            missing_players.append(player['name'])
    
    return missing_players
    
    

Again, the following script is commented out because the final dataframes were saved to csv files after being properly generated. The next cell was used to find missing players for each season, which can be found in the `missing_urls_2019` and `missing_urls_2020` dictionaries in the subsequent cell. 

In [23]:
'''
# Get 2019 missing players
missing_players_2019 = find_missing_players(df_basic_stats_2019, df_gamelogs_2019)
print('2019 missing players:')
print(missing_players_2019)

# Get 2020 missing players
missing_players_2020 = find_missing_players(df_basic_stats_2020, df_gamelogs_2020)
print('2020 missing players:')
print(missing_players_2020)
'''

"\n# Get 2019 missing players\nmissing_players_2019 = find_missing_players(df_basic_stats_2019, df_gamelogs_2019)\nprint('2019 missing players:')\nprint(missing_players_2019)\n\n# Get 2020 missing players\nmissing_players_2020 = find_missing_players(df_basic_stats_2020, df_gamelogs_2020)\nprint('2020 missing players:')\nprint(missing_players_2020)\n"

The following cell was used to collect the game log data for players missing from the original 2019 and 2020 game logs dataframes because of formatting issues with URLs. 

In [24]:
#manually write in missing players due to formatting issues with urls

# Manually enter player names and their corresponding urls

missing_urls_2019 = {'J.T. Realmuto' : 'https://baseballsavant.mlb.com/savant-player/j-t-realmuto-592663?stats=gamelogs-r-hitting-statcast&season=2019&gamelogs_event=&gamelogs_direction=&gamelogs_pitchtypes=&gamelogs_view=statcastGameLogs', 
                    'J.D. Martinez' : 'https://baseballsavant.mlb.com/savant-player/j-d-martinez-502110?stats=gamelogs-r-hitting-statcast&season=2019&gamelogs_event=&gamelogs_direction=&gamelogs_pitchtypes=&gamelogs_view=statcastGameLogs', 
                    'Josh Bell' : 'https://baseballsavant.mlb.com/savant-player/josh-bell-605137?stats=gamelogs-r-hitting-statcast&season=2019&gamelogs_event=&gamelogs_direction=&gamelogs_pitchtypes=&gamelogs_view=statcastGameLogs'}

missing_urls_2020 = {'J.T. Realmuto' : 'https://baseballsavant.mlb.com/savant-player/j-t-realmuto-592663?stats=gamelogs-r-hitting-statcast&season=2020&gamelogs_event=&gamelogs_direction=&gamelogs_pitchtypes=&gamelogs_view=statcastGameLogs', 
                    'J.D. Martinez' : 'https://baseballsavant.mlb.com/savant-player/j-d-martinez-502110?stats=gamelogs-r-hitting-statcast&season=2020&gamelogs_event=&gamelogs_direction=&gamelogs_pitchtypes=&gamelogs_view=statcastGameLogs', 
                    'Josh Bell' : 'https://baseballsavant.mlb.com/savant-player/josh-bell-605137?stats=gamelogs-r-hitting-statcast&season=2020&gamelogs_event=&gamelogs_direction=&gamelogs_pitchtypes=&gamelogs_view=statcastGameLogs',
                    'J.D. Davis' : 'https://baseballsavant.mlb.com/savant-player/j-d-davis-605204?stats=gamelogs-r-hitting-statcast&season=2020&gamelogs_event=&gamelogs_direction=&gamelogs_pitchtypes=&gamelogs_view=statcastGameLogs',
                    'J.P. Crawford' : 'https://baseballsavant.mlb.com/savant-player/j-p-crawford-641487?stats=gamelogs-r-hitting-statcast&season=2020&gamelogs_event=&gamelogs_direction=&gamelogs_pitchtypes=&gamelogs_view=statcastGameLogs'}


missing_dict = {2019 : missing_urls_2019, 
                2020 : missing_urls_2020}

'''
for year, missing_urls in missing_dict.items():
    # Iterate over each missing players
    for name, player_url in missing_urls.items():
        # Scrape with selenium
        driver = webdriver.Chrome()
        driver.implicitly_wait(15)
        driver.get(player_url)
        gamelog_table = driver.find_element_by_id('gamelogs_statcast')
        
        # Get missing player's game logs 
        player_game_logs = gamelog_table.find_elements_by_class_name('default-table-row')
        df_player_gamelogs_clean = clean_player_gamelogs(player_game_logs, name)

        # Add player game logs to the correct year
        if (year == 2019):
            df_gamelogs_2019 = df_gamelogs_2019.append(df_player_gamelogs_clean, ignore_index=True)
        else:
            df_gamelogs_2020 = df_gamelogs_2020.append(df_player_gamelogs_clean, ignore_index=True)
'''      

"\nfor year, missing_urls in missing_dict.items():\n    # Iterate over each missing players\n    for name, player_url in missing_urls.items():\n        # Scrape with selenium\n        driver = webdriver.Chrome()\n        driver.implicitly_wait(15)\n        driver.get(player_url)\n        gamelog_table = driver.find_element_by_id('gamelogs_statcast')\n        \n        # Get missing player's game logs \n        player_game_logs = gamelog_table.find_elements_by_class_name('default-table-row')\n        df_player_gamelogs_clean = clean_player_gamelogs(player_game_logs, name)\n\n        # Add player game logs to the correct year\n        if (year == 2019):\n            df_gamelogs_2019 = df_gamelogs_2019.append(df_player_gamelogs_clean, ignore_index=True)\n        else:\n            df_gamelogs_2020 = df_gamelogs_2020.append(df_player_gamelogs_clean, ignore_index=True)\n"

Once populated with each qualified player, the 2019 and 2020 game logs dataframes were saved to csvs. 

In [25]:
#save full player gamelogs

#df_gamelogs_2019.to_csv('gamelogs2019.csv')
#df_gamelogs_2020.to_csv('gamelogs2020.csv')

In [26]:
# Read the DataFrame of gamelogs saved on the last run of the previous script: 
df_gamelogs_2019 = pd.read_csv('gamelogs2019.csv')
del df_gamelogs_2019['Unnamed: 0']
df_gamelogs_2019.head()

df_gamelogs_2020 = pd.read_csv('gamelogs2020.csv')
del df_gamelogs_2020['Unnamed: 0']
df_gamelogs_2020.head()

,exit_velo,launch_angle,player,result,hit_type,is_hit
0,69.9,49.0,David Peralta,field_out,out,0
1,62.8,36.0,David Peralta,field_out,out,0
2,91.4,7.0,David Peralta,single,single,1
3,43.4,16.0,David Peralta,field_out,out,0
4,112.1,16.0,David Peralta,double,double,1


In [27]:
# Test there are no missing players 
assert find_missing_players(df_basic_stats_2019, df_gamelogs_2019) == []
assert find_missing_players(df_basic_stats_2020, df_gamelogs_2020) == []

To prepare for machine learning applications, we add two columns to the gamelogs dataframes. The first is `hit_type` and is a simpler version of `result`, which treats all outs the same (rather than `field_out`, `sacrifice_fly`, `sacrifice_bunt`, etc), but keeps `single`'s, `double`'s, and `triple`'s the same.

The second column is called `is_hit` and contains a 1 if the result was a hit, and 0 otherwise. 

In [28]:
def add_hit_type_is_hit_cols(df_gamelogs):
    ''' Adds hit_type and is_hit columns to the given gamelogs dataframe
    
    Args:
        df_gamelogs (pd.DataFrame): contains statcast gamelogs for all qualified hitters in a season
        
    Returns:
        df_gamelogs (pd.DataFrame): df_gamelogs including hit_type and is_hit columms
    
    '''

    # Empty columns
    hit_type = []
    is_hit = []
    
    # For each batted ball event, update hit_type and is_hit rows
    for idx, row in df_gamelogs.iterrows():
        # If the result was a hit, keep hit_type the same, add 1 to is_hit
        if row['result'] == 'single' or row['result'] == 'double' or row['result'] == 'triple':
            hit_type.append(row['result'])
            is_hit.append(1)
        # If the result was any type of out, add 'out' to hit_type, 0 to is_hit
        else:
            hit_type.append('out')
            is_hit.append(0)

    # Create hit_type and is_hit series objects, add them as columns to df_gamelogs       
    hit_type_series = pd.Series(hit_type)
    is_hit_series = pd.Series(is_hit)
    df_gamelogs['hit_type'] = hit_type_series
    df_gamelogs['is_hit'] = is_hit_series
    
    return df_gamelogs

In [29]:
# Add columns to df_gamelogs_2019
df_gamelogs_2019 = add_hit_type_is_hit_cols(df_gamelogs_2019)

# Add columns to df_gamelogs_2020
df_gamelogs_2020 = add_hit_type_is_hit_cols(df_gamelogs_2020)

# Show updated dataframes
print('2019 Gamelogs')
display(df_gamelogs_2019.head())
print('2020 Gamelogs')
display(df_gamelogs_2020.head())

2019 Gamelogs


,exit_velo,launch_angle,player,result,hit_type,is_hit
0,87.6,13.0,Albert Pujols,single,single,1
1,84.7,44.0,Albert Pujols,field_out,out,0
2,99.4,28.0,Albert Pujols,field_out,out,0
3,95.3,17.0,Albert Pujols,field_out,out,0
4,84.8,-40.0,Albert Pujols,field_out,out,0


2020 Gamelogs


,exit_velo,launch_angle,player,result,hit_type,is_hit
0,69.9,49.0,David Peralta,field_out,out,0
1,62.8,36.0,David Peralta,field_out,out,0
2,91.4,7.0,David Peralta,single,single,1
3,43.4,16.0,David Peralta,field_out,out,0
4,112.1,16.0,David Peralta,double,double,1


In [30]:
# Save updated gamelogs
df_gamelogs_2019.to_csv('gamelogs2019.csv')
df_gamelogs_2020.to_csv('gamelogs2020.csv')

## Fangraphs Custom 2020 Statistics
Finally, we used custom filters on [Fangraphs](https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=y&type=8&season=2020&month=0&season1=2020&ind=0) to construct a data set of statistics for the 2020 season that include statistics such as:
- `BB%`: walk rate (walks per plate appearance)
- `K%`: strikeout rate (strikeouts per plate appearance)
- `LD%`: line drive percentage (line drives per balls in play)
- `FB%`: fly ball percentage (fly balls per balls in play)
- `GB%`: ground ball percentage (ground balls per balls in play)
- `HR/FB`: home run to fly ball ratio (homeruns per fly balls) 
- and more
These statistics will be used to compare players whose estimated BABIPs are higher, lower, or equal to their true BABIP in order to investigate how certain tendencies of hitters might lead them to be more or less likely to get a hit on a ball in play. 

In [31]:
df_custom_2020 = pd.read_csv('fangraphs_custom_2020.csv')
df_custom_2020.head()

,Name,Team,G,PA,HR,R,RBI,SB,BB%,K%,...,IFH%,BUH%,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,playerid
0,Michael Lorenzen,CIN,23,1,0,2,0,0,0.0%,0.0%,...,0.0%,0.0%,NaN,NaN,0.0,NaN,NaN,0.0,NaN,14843
1,Luis Campusano,SDP,1,4,1,2,1,0,0.0%,50.0%,...,0.0%,0.0%,101.1,21.6,0.0,0.0%,101.1,1.0,100.0%,22217
2,Rafael Marchan,PHI,3,9,1,3,3,0,11.1%,22.2%,...,0.0%,0.0%,92.2,10.5,1.0,16.7%,99.6,3.0,50.0%,21646
3,Billy McKinney,TOR,2,3,0,1,0,0,0.0%,0.0%,...,0.0%,0.0%,89.1,23.7,0.0,0.0%,104.8,1.0,33.3%,15654
4,Travis Blankenhorn,MIN,1,4,0,0,0,0,0.0%,0.0%,...,0.0%,0.0%,89.3,41.2,1.0,33.3%,108.8,1.0,33.3%,18395


In [32]:
# Remove players that did not qualify in 2020
list_qual = list(df_basic_stats_2020['name'])
df_custom_qual_2020 = df_custom_2020.loc[df_custom_2020['Name'].isin(list_qual)]

# Add AJ Pollock 
df_custom_qual_2020 = df_custom_qual_2020.append(df_custom_2020.loc[df_custom_2020['Name'] == 'A.J. Pollock',:])


In [33]:
df_custom_qual_2020

,Name,Team,G,PA,HR,R,RBI,SB,BB%,K%,...,IFH%,BUH%,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,playerid
7,Juan Soto,WSN,47,196,13,39,37,6,20.9%,14.3%,...,6.2%,0.0%,92.1,4.2,22.0,17.5%,113.3,65.0,51.6%,20123
11,Freddie Freeman,ATL,60,262,13,51,53,2,17.2%,14.1%,...,5.4%,0.0%,92.4,17.2,26.0,14.7%,109.3,96.0,54.2%,5361
12,Marcell Ozuna,ATL,60,267,18,38,56,0,14.2%,22.5%,...,3.2%,0.0%,93.0,16.4,26.0,15.4%,115.6,91.0,53.8%,10324
16,DJ LeMahieu,NYY,50,216,10,41,27,3,8.3%,9.7%,...,4.0%,0.0%,91.3,2.3,5.0,2.9%,109.5,80.0,45.7%,9874
22,Jose Ramirez,CLE,58,254,17,45,46,10,12.2%,16.9%,...,11.3%,100.0%,88.7,23.2,18.0,10.2%,114.3,62.0,35.0%,13510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,Evan White,SEA,54,202,8,19,26,1,8.9%,41.6%,...,11.9%,0.0%,91.7,13.7,14.0,14.1%,112.8,52.0,52.5%,20078
421,Eduardo Escobar,ARI,54,222,4,22,20,1,6.8%,18.5%,...,1.7%,0.0%,88.6,18.1,9.0,5.5%,106.7,52.0,31.7%,6153
423,Javier Baez,CHC,59,235,8,27,24,3,3.0%,31.9%,...,9.6%,100.0%,89.4,10.3,12.0,8.1%,116.0,60.0,40.3%,12979
427,Nicky Lopez,KCR,56,192,1,15,13,0,9.4%,21.4%,...,4.3%,0.0%,84.9,1.4,2.0,1.5%,104.6,35.0,26.7%,19339


#### Convert Percentages to Decimals
Then, in order to perform statistical analyses, we convert percentages in relevant columns to their decimal equivalents. 

In [34]:
def percent_to_dec(percent):
    ''' Converts the given percentage to a decimal
    
    Args:
        percent (str): percentages followed by '%'
    
    Returns:
        decimal (pd.Series): converted decimal values
    '''
    str_dec = percent[0:len(percent)-1]
    decimal = float(str_dec) / 100
        
    return decimal

In [35]:
# Convert columns from percentage strings to decimals
x_feat_list = ['BB%', 'K%', 'LD%', 'GB%', 'FB%', 'HR/FB', 'Barrel%']

# Convert relevant columns in x_feat_list to decimals
for feat in x_feat_list:
    feat_col = df_custom_qual_2020[feat].apply(percent_to_dec)
    df_custom_qual_2020[feat] = feat_col

df_custom_qual_2020 = df_custom_qual_2020.reset_index()
df_custom_qual_2020

,index,Name,Team,G,PA,HR,R,RBI,SB,BB%,...,IFH%,BUH%,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,playerid
0,7,Juan Soto,WSN,47,196,13,39,37,6,0.209,...,6.2%,0.0%,92.1,4.2,22.0,0.175,113.3,65.0,51.6%,20123
1,11,Freddie Freeman,ATL,60,262,13,51,53,2,0.172,...,5.4%,0.0%,92.4,17.2,26.0,0.147,109.3,96.0,54.2%,5361
2,12,Marcell Ozuna,ATL,60,267,18,38,56,0,0.142,...,3.2%,0.0%,93.0,16.4,26.0,0.154,115.6,91.0,53.8%,10324
3,16,DJ LeMahieu,NYY,50,216,10,41,27,3,0.083,...,4.0%,0.0%,91.3,2.3,5.0,0.029,109.5,80.0,45.7%,9874
4,22,Jose Ramirez,CLE,58,254,17,45,46,10,0.122,...,11.3%,100.0%,88.7,23.2,18.0,0.102,114.3,62.0,35.0%,13510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,412,Evan White,SEA,54,202,8,19,26,1,0.089,...,11.9%,0.0%,91.7,13.7,14.0,0.141,112.8,52.0,52.5%,20078
138,421,Eduardo Escobar,ARI,54,222,4,22,20,1,0.068,...,1.7%,0.0%,88.6,18.1,9.0,0.055,106.7,52.0,31.7%,6153
139,423,Javier Baez,CHC,59,235,8,27,24,3,0.030,...,9.6%,100.0%,89.4,10.3,12.0,0.081,116.0,60.0,40.3%,12979
140,427,Nicky Lopez,KCR,56,192,1,15,13,0,0.094,...,4.3%,0.0%,84.9,1.4,2.0,0.015,104.6,35.0,26.7%,19339


In [36]:
# Save updated custom 2020 stats
df_custom_qual_2020.to_csv('custom_stats_qual_2020.csv')